# Problem Statement
Sometimes, some studies don't get segmented after running the `FASTEST_segmentations` DAG, likely because the original itksnap files were wrongly labeled.
In such cases, and when it's urgent to get the segmentations numbers, run this notebook.

# Solutions
Get segmentation numbers for specified files in FASTEST. The code was copied from the `FASTEST_segmentations` DAG, with some changes. It should not update the database.

# Code

In [ ]:
import json
from datetime import datetime
from pathlib import Path
import AMBRA_Backups
import nibabel as nib
import numpy as np

db_name = "FASTEST"

segmentation_dir = Path("/Volumes/FASTEST/Processing/")
assert segmentation_dir.exists()

In [ ]:
# Define the folders with files that need segmentations
need_segmentation = [
    Path(
        "/Volumes/FASTEST/Processing/1616/MR_MRI_BRAIN_WO_FOLLOW_UP_3a5de144b716_2024_12_22"
    )
]

In [ ]:
def read_info(directory):
    info_path = Path(directory).joinpath("info.json")
    assert info_path.exists()

    with open(info_path, "r") as fopen:
        data = json.load(fopen)

    return data

In [ ]:
def get_id_study(db_obj, study_uid):
    res = db_obj.run_select_query(
        "SELECT id FROM studies WHERE study_uid=%s", record=(study_uid,)
    )
    return res[0][0]

In [ ]:
def get_roi_info(roi_path):
    info = {"path": roi_path}

    img = nib.load(roi_path)
    header = img.header

    vox_volume = header["pixdim"][1] * header["pixdim"][2] * header["pixdim"][3]
    info["vox_volume"] = vox_volume
    info["pixdim"] = header["pixdim"][1:4]
    info["dim"] = header["dim"][1:4]
    info["roi_volumes"] = []

    info["timepoint"] = None
    if "_BL" in roi_path.name:
        info["timepoint"] = "BL"
    elif "_FU" in roi_path.name:
        info["timepoint"] = "FU"
    elif "_Unsched" in roi_path.name:
        info["timepoint"] = "US"
    elif "_US" in roi_path.name:
        info["timepoint"] = "US"
    elif "_UN" in roi_path.name:
        info["timepoint"] = "US"

    data = img.get_fdata()
    info["non_zero_volume"] = np.where(data > 0)[0].shape[0] * vox_volume
    for index in np.unique(np.unique(data)):
        if index == 0:
            continue
        n_voxels = np.where(data == index)[0].shape[0]
        volume = n_voxels * vox_volume
        roi_info = {"index": index, "n_voxels": n_voxels, "volume": volume}
        info["roi_volumes"].append(roi_info)

    return info

In [ ]:
def add_segmentation(db_obj, roi_path, id_study):
    roi_path = Path(roi_path)
    roi_info = get_roi_info(roi_path)

    try:
        roi1_volume = float(
            [
                this["volume"]
                for this in roi_info["roi_volumes"]
                if int(this["index"]) == 1
            ][0]
        )
    except IndexError:
        roi1_volume = 0
    try:
        roi2_volume = float(
            [
                this["volume"]
                for this in roi_info["roi_volumes"]
                if int(this["index"]) == 2
            ][0]
        )
    except IndexError:
        roi2_volume = 0
    try:
        roi3_volume = float(
            [
                this["volume"]
                for this in roi_info["roi_volumes"]
                if int(this["index"]) == 3
            ][0]
        )
    except IndexError:
        roi3_volume = 0

    info_dic = {
        "roi_path": str(roi_path),
        "id_study": id_study,
        "timepoint": roi_info["timepoint"],
        "roi1_volume": roi1_volume,
        "roi2_volume": roi2_volume,
        "roi3_volume": roi3_volume,
        "n_rois": len(roi_info["roi_volumes"]),
        "vox_dim_x": float(roi_info["pixdim"][0]),
        "vox_dim_y": float(roi_info["pixdim"][1]),
        "vox_dim_z": float(roi_info["pixdim"][2]),
        "stats_added": datetime.now(),
    }
    print("#######################")
    print("info_dict: ")
    print(info_dic)
    print("----------------")
    print("relevant info:")
    print(f"""
        roi1_volume_{info_dic["timepoint"]}: {info_dic["roi1_volume"] / 1000}
        """)

In [ ]:
def add_segmentations(study_dirs):
    db = AMBRA_Backups.database.Database(db_name)

    for study_dir in study_dirs:
        study_info = read_info(study_dir)
        print("\t", study_dir.name)

        nii_files = study_dir.glob("*WS*.nii*")
        for nii_file in nii_files:
            try:
                img = nib.load(nii_file)
                img_data = img.get_fdata()
            except Exception:
                continue

            if np.unique(img_data).shape[0] < 4:
                print("\t\t", nii_file.name)
                print("\t\t\tN unique: ", np.unique(img_data).shape[0])
                id_study = get_id_study(db, study_info["study_uid"])
                print(id_study)

                add_segmentation(db, nii_file, id_study)

In [ ]:
add_segmentations(need_segmentation)